# Install

# Import

In [1]:
import pandas as pd
import re
import logging
logging.getLogger('pdfminer.pdfinterp').setLevel(logging.ERROR)
logging.getLogger('pdfminer.layout').setLevel(logging.ERROR)
from modules.schedule import ITR1_SECTIONS

# fn

In [2]:
from modules.extract_logic import clean_row,extract_sections,apply_dynamic_headers,is_empty_row_specific

# Code

In [3]:
input_file_path = 'OUTPUT/ ITR 1-2 AY 2025-26.pdf'
output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
# excel_file_path = "output.xlsx"
config_path = "config/ITR1_header.json"
itr1 = ITR1_SECTIONS(input_file_path, output_file_path, config_path)
itr1

/home/simelabs/fun-pdf-to-excel/modules/extract_logic.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace("", np.nan)  # Use np.nan for dropna


In [4]:
# assert False

In [5]:
# itr1.export_to_excel(excel_file_path)

In [6]:
# itr1.start_pattern

In [7]:
# itr1.end_pattern

In [8]:
# itr1.extracted_data[33:34]

In [9]:
df_temp = pd.DataFrame(itr1.extracted_data[147:188])
# df_temp = clean_row(df_temp)
# df_temp = apply_dynamic_headers(df_temp, itr1.config,"ScheduleEA10_13A")
# df_temp

In [10]:
cols="SCHEDULE_80D"
print(itr1.start_pattern.get(cols,"<No found>"))
print(itr1.end_pattern.get(cols,"<No found>"))
print(itr1.hdr_row_map.get(cols,"<No found>"))

^SCHEDULE 80D$
Eligible Amount of Deduction
['1']


In [11]:
extract_sections(itr1.extracted_data[147:188], itr1.start_pattern,itr1.end_pattern,itr1.hdr_row_map)

{'SCHEDULE_80D': {'start': 2, 'end': 40}}

In [12]:
cols="SCHEDULE_80D"
itr1.sections.get(cols,"<No found>")

{'start': 149, 'end': 187}

In [13]:
df_temp = itr1.get_section(cols)
# display(df_temp)

In [14]:
# df_temp.to_excel("del.xlsx",index=False)

In [15]:
# assert False

In [16]:
itr1.acknowledgement,itr1.dof

('560199660070925', '07-Sep-2025')

In [17]:
df_EXEMPT_INCOME= itr1.get_section("EXEMPT_INCOME")
display(df_EXEMPT_INCOME)

,RowNumber,Nature_of_Income,Description,Amount
0,(1),(2),(3),(4)
1,Total Amount,None,None,0


In [18]:
df_LTCG112A= itr1.get_section("LTCG112A")
display(df_LTCG112A)

,0,3,9
0,i,Total sale consideration,0
1,ii,Total cost of acquisition,0
2,iii,Long term capital gains as per sec 112A,0


In [40]:
def header_to_data_row(df,hdr):
    header_as_list = df.columns.tolist()
    new_cols = list(range(len(header_as_list)))
    header_row_df = pd.DataFrame([header_as_list], columns=new_cols)
    df.columns = new_cols
    hdr_row_df = pd.DataFrame([hdr], columns=list(range(len(hdr))))
    df_processed = pd.concat([hdr_row_df,header_row_df], ignore_index=True)
    df_processed = pd.concat([df_processed, df.copy()], ignore_index=True)
    return df_processed

df1 = pd.DataFrame({'Col_A': ['X1', 'X2'], 'Col_B': ['Y1', 'Y2'], 'Col_C': ['Z1', 'Z2']})
df2 = pd.DataFrame({'Col_A': ['A3', 'A4'], 'Col_Y': ['B3', 'B4'], 'Col_Z': ['C3', 'C4'], 'Col_D': ['D3', 'D4']})

hdr = [itr1.acknowledgement,itr1.dof]
df_result1 = header_to_data_row(df_EXEMPT_INCOME,hdr)
df_result2 = header_to_data_row(df_LTCG112A,hdr)

df_final_combined = pd.concat([df_result1, df_result2], ignore_index=True)
df_final_combined
# print(df_final_combined.to_markdown(index=False))

,0,1,2,3
0,560199660070925,07-Sep-2025,NaN,NaN
1,RowNumber,Nature_of_Income,Description,Amount
2,(1),(2),(3),(4)
3,Total Amount,None,None,0
4,560199660070925,07-Sep-2025,NaN,NaN
5,0,3,9.0,NaN
6,i,Total sale consideration,0,NaN
7,ii,Total cost of acquisition,0,NaN
8,iii,Long term capital gains as per sec 112A,0,NaN


In [20]:
header_row_df

,0,1
0,560199660070925,07-Sep-2025


In [21]:
# df_PART_C= itr1.get_section("PART_C")
# display(df_PART_C)

In [22]:
# df_PART_D= itr1.get_section("PART_D")
# display(df_PART_D)

In [23]:
df_PART_E= itr1.get_section("PART_E")
display(df_PART_E)

,RowNumber,IFSCodeoftheBank,NameoftheBank,AccountNumber,Typeofaccount,SelectAccountforRefundCredit
0,(1),(2),(3),(4),(5),(6)
1,1,BARB0NAINIT,BANK OF BARODA,06090100019492,Savings Account,NaN
2,2,KNSB0010010,KURMANCHAL NAGAR\nSAHKARI BANK LTD,010010100002700,Savings Account,NaN


In [24]:
df_EA10_13A= itr1.get_section("ScheduleEA10_13A")
display(df_EA10_13A)

,Placeofwork,ActualHRAreceived(A),ActualRentpaid,DetailsofSalaryaspersection17(1),BasicSalary,DearnessAllowance,Actualrentpaid-10%ofsalary(B)(3-10%of4),50%/40%ofsalary,EligibleExemptAllowanceu/s10(13A)
0,(1),(2),(3),(4),(4a),(4b),(5),(6),(7)
1,NaN,0,0,0,0,0,0,0,0


In [25]:
df_80u = itr1.get_section("SECTION_80U")
display(df_80u)

,RowNumber,NatureOfDisability,TypeOfDisability,AmountOfDeduction,Form10IA_AckNum,UDID_Num
0,(1),(2a),(2b),(3),(4),(5)
1,1,NaN,NaN,0,NaN,NaN


In [26]:
df_80dd = itr1.get_section("SECTION_80DD")
display(df_80dd)

,RowNumber,NatureOfDisability,TypeOfDisability,AmountOfDeduction,DependentType,DependentPan,DependentAadhaar,Form10IA_AckNum,UDID_Num
0,(1),(2a),(2b),(3),(4),(5),(6),(7),(8)
1,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [27]:
df_80G_A= itr1.get_section("SCHEDULE_80G_A")
display(df_80G_A)

,RowNumber,NameofDonee,Address,City,StateCode,PinCode,PANofDonee,TotDon100PercentCash,TotDon100PercentOtherMode,TotDon100Percent,TotEligibleDon100Percent
0,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),(11)
1,Total A,None,None,None,None,None,None,0,0,0,0


In [28]:
df_80G_B= itr1.get_section("SCHEDULE_80G_B")
display(df_80G_B)

,RowNumber,NameofDonee,Address,City,StateCode,PinCode,PANofDonee,TotDon50PercentNoApprReqdCash,TotDon50PercentNoApprReqdOtherMode,TotDon50PercentNoApprReqd,TotEligibleDon50Percent
0,(1),(2),(3),(4),(5),(6),(7),(9),(10),(11),(12)
1,Total B,None,None,None,None,None,None,0,0,0,0


In [29]:
df_80G_C= itr1.get_section("SCHEDULE_80G_C")
display(df_80G_C)

,RowNumber,NameofDonee,Address,City,StateCode,PinCode,PANofDonee,TotDon100PercentApprReqdCash,TotDon100PercentApprReqdOtherMode,TotDon100PercentApprReqd,TotEligibleDon100PercentApprReqd
0,(1),(2),(3),(4),(5),(6),(7),(9),(10),(11),(12)
1,Total C,None,None,None,None,None,None,0,0,0,0


In [30]:
df_80G_D= itr1.get_section("SCHEDULE_80G_D")
display(df_80G_D)

,RowNumber,NameofDonee,Address,City,StateCode,PinCode,PANofDonee,ARN(DonationReferenceNumber),TotDon100PercentApprReqdCash,TotDon100PercentApprReqdOtherMode,TotDon100PercentApprReqd,TotEligibleDon100PercentApprReqd
0,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),(11),(12)
1,Total D,None,None,None,None,None,None,None,0,0,0,0


In [31]:
df_80GGA= itr1.get_section("SCHEDULE_80GGA")
display(df_80GGA)

,RowNumber,RelevantClause,NameofDonee,Address,City,StateCode,PinCode,PANofDonee,TotalDonationAmtCash80GGA,TotalDonationAmtOtherMode80GGA,TotalDonationsUs80GGA,TotalEligibleDonationAmt80GGA
0,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),(11),(12)
1,Total Donation,None,None,None,None,None,None,None,0,0,0,0


In [32]:
df_80GGC= itr1.get_section("SCHEDULE_80GGC")
display(df_80GGC)

,RowNumber,Date,TotalDonationAmtCash80GGC,TotalDonationAmtOtherMode80GGC,TotalDonationsUs80GGC,TotalEligibleDonationAmt80GGC,TransactionReference,IFSCcode
0,(1),(2),(3),(4),(5),(6),(7),(8)
1,Total Contribution,None,0,0,0,0,NaN,NaN


In [33]:
df_it= itr1.get_section("SCHEDULE_IT")
display(df_it)

,RowNumber,BSRCode,DateofDeposit,SerialNumberofChallan,TaxPaid
0,(1),(2),(3),(4),(5)
1,1,0300051,06/09/2025,30086,"5,225"
2,Total,None,None,None,"5,225"


In [34]:
df_TDS1= itr1.get_section("SCHEDULE_TDS1")
display(df_TDS1)

,RowNumber,TANofTheDeductor,NameofDeductor,IncomeChargeableSalaries,TaxPaid
0,(1),(2),(3),(4),(5)
1,1,MRTK01232A,KURMANCHAL NAGAR SAHKARI\nBANK LTD,"19,31,428","2,59,000"
2,Total,None,None,None,"2,59,000"


In [35]:
df_TDS2= itr1.get_section("SCHEDULE_TDS2")
display(df_TDS2)

,RowNumber,TANofTheDeductor,NameofDeductor,SectionUnderTDSdeducted,GrossReceipt,YearofTaxDeduction,TaxDeducted,TDSCreditOutof(6)ClaimedThisYear
0,(1),(2),(3a),(3b),(4),(5),(6),(7)
1,Total,None,None,None,None,None,None,0


In [36]:
df_TDS3= itr1.get_section("SCHEDULE_TDS3")
display(df_TDS3)

,RowNumber,PANoftheTenant,AadhaarNumofTenant,NameoftheTenant,SectionUnderTDSdeducted,GrossReceipt,YearofTaxDeduction,TaxDeducted,TDSCreditOutof(7)ClaimedThisYear
0,(1),(2),(3),(4a),(4b),(5),(6),(7),(8)
1,Total,None,None,None,None,None,None,None,0


In [37]:
df_TCS= itr1.get_section("SCHEDULE_TCS")
display(df_TCS)

,RowNumber,TaxCollectionAccountNumberoftheCollector,Name of theCollector,GrossPaymentTaxCollection,YearofTaxCollection,TaxCollected,TDSCreditOutof(6)ClaimedThisYear
0,(1),(2),(3),(4),(5),(6),(7)
1,Total,None,None,None,None,None,0
